# 02 — Stress Indicators
Compute demand-supply ratio, reserve proxy, and stress z-score for sector `total`, then generate figures and a top-10 stress table.


In [ ]:
import csv
import math
import statistics
from datetime import datetime
from pathlib import Path

DATA_DIR = Path("data/processed")
REPORTS_DIR = Path("reports")
FIG_DIR = REPORTS_DIR / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

def read_csv(path):
    with open(path, newline="", encoding="utf-8") as f:
        return list(csv.DictReader(f))

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

def month_range(start, end):
    months = []
    cur = datetime(start.year, start.month, 1)
    while cur <= end:
        months.append(cur.strftime("%Y-%m-%d"))
        if cur.month == 12:
            cur = datetime(cur.year + 1, 1, 1)
        else:
            cur = datetime(cur.year, cur.month + 1, 1)
    return months


In [ ]:
def write_svg_line_chart(path, title, x_labels, series_dict, y_label="Value"):
    width, height = 1100, 420
    ml, mr, mt, mb = 70, 20, 50, 70
    pw, ph = width - ml - mr, height - mt - mb
    values = [v for vals in series_dict.values() for v in vals if v is not None]
    y_min, y_max = min(values), max(values)
    if y_max == y_min:
        y_max += 1.0
    def sx(i):
        return ml + (i/(len(x_labels)-1))*pw if len(x_labels) > 1 else ml
    def sy(v):
        return mt + (1 - (v - y_min)/(y_max-y_min))*ph
    colors = ["#1f77b4", "#d62728", "#2ca02c", "#9467bd"]
    lines = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">',
             f'<text x="{width/2}" y="24" text-anchor="middle" font-size="18" font-family="Arial">{title}</text>',
             f'<line x1="{ml}" y1="{mt}" x2="{ml}" y2="{mt+ph}" stroke="#333"/>',
             f'<line x1="{ml}" y1="{mt+ph}" x2="{ml+pw}" y2="{mt+ph}" stroke="#333"/>',
             f'<text x="16" y="{mt+ph/2}" transform="rotate(-90 16,{mt+ph/2})" font-size="12" font-family="Arial">{y_label}</text>']
    for i in range(6):
        yv = y_min + (y_max-y_min)*i/5
        y = sy(yv)
        lines.append(f'<line x1="{ml}" y1="{y}" x2="{ml+pw}" y2="{y}" stroke="#eee"/>')
        lines.append(f'<text x="{ml-8}" y="{y+4}" text-anchor="end" font-size="10" font-family="Arial">{yv:.2f}</text>')
    tick_step = max(1, len(x_labels)//12)
    for i, label in enumerate(x_labels):
        if i % tick_step == 0:
            x = sx(i)
            lines.append(f'<line x1="{x}" y1="{mt+ph}" x2="{x}" y2="{mt+ph+5}" stroke="#333"/>')
            lines.append(f'<text x="{x}" y="{mt+ph+18}" transform="rotate(45 {x},{mt+ph+18})" font-size="9" font-family="Arial">{label[:7]}</text>')
    for idx, (name, vals) in enumerate(series_dict.items()):
        color = colors[idx % len(colors)]
        segment = []
        for i, v in enumerate(vals):
            if v is None:
                if len(segment) > 1:
                    lines.append(f'<polyline fill="none" stroke="{color}" stroke-width="2" points="{" ".join(segment)}"/>')
                segment = []
            else:
                segment.append(f"{sx(i):.1f},{sy(v):.1f}")
        if len(segment) > 1:
            lines.append(f'<polyline fill="none" stroke="{color}" stroke-width="2" points="{" ".join(segment)}"/>')
        ly = mt + 16*idx
        lines.append(f'<rect x="{ml+pw-170}" y="{ly-10}" width="10" height="10" fill="{color}"/>')
        lines.append(f'<text x="{ml+pw-154}" y="{ly}" font-size="11" font-family="Arial">{name}</text>')
    lines.append('</svg>')
    Path(path).write_text("\n".join(lines), encoding="utf-8")

def write_svg_bar_chart(path, title, labels, values, y_label="Value"):
    width, height = 1100, 420
    ml, mr, mt, mb = 70, 20, 50, 120
    pw, ph = width - ml - mr, height - mt - mb
    y_min = min(0.0, min(values))
    y_max = max(values)
    if y_max == y_min:
        y_max += 1.0
    def sx(i):
        bw = pw/len(labels)
        return ml + i*bw + bw*0.15
    def sw():
        return (pw/len(labels))*0.7
    def sy(v):
        return mt + (1 - (v-y_min)/(y_max-y_min))*ph
    lines=[f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">',
           f'<text x="{width/2}" y="24" text-anchor="middle" font-size="18" font-family="Arial">{title}</text>',
           f'<line x1="{ml}" y1="{mt}" x2="{ml}" y2="{mt+ph}" stroke="#333"/>',
           f'<line x1="{ml}" y1="{sy(0)}" x2="{ml+pw}" y2="{sy(0)}" stroke="#333"/>']
    for i,v in enumerate(values):
        x=sx(i); y=sy(max(v,0)); h=abs(sy(0)-sy(v))
        color="#d62728" if v>=0 else "#1f77b4"
        lines.append(f'<rect x="{x}" y="{y}" width="{sw()}" height="{h}" fill="{color}" opacity="0.8"/>')
        lx=x+sw()/2
        lines.append(f'<text x="{lx}" y="{mt+ph+15}" transform="rotate(45 {lx},{mt+ph+15})" font-size="9" font-family="Arial">{labels[i][:7]}</text>')
    lines.append(f'<text x="16" y="{mt+ph/2}" transform="rotate(-90 16,{mt+ph/2})" font-size="12" font-family="Arial">{y_label}</text>')
    lines.append('</svg>')
    Path(path).write_text("\n".join(lines), encoding="utf-8")


In [ ]:
supply_rows = [r for r in read_csv(DATA_DIR / "electricity_supply_clean.csv") if r["sector"] == "total"]
cons_rows = [r for r in read_csv(DATA_DIR / "electricity_consumption_clean.csv") if r["sector"] == "total"]
supply = {r["date"]: float(r["supply"]) for r in supply_rows}
cons = {r["date"]: float(r["consumption"]) for r in cons_rows}
months = sorted(set(supply).intersection(cons))

records = []
for d in months:
    s = supply[d]
    c = cons[d]
    ratio = c / s if s else math.nan
    reserve = s - c
    records.append({"date": d, "supply": s, "consumption": c, "demand_supply_ratio": ratio, "reserve_proxy": reserve})

ratios = [r["demand_supply_ratio"] for r in records]
mu = statistics.mean(ratios)
sd = statistics.pstdev(ratios)
for r in records:
    r["stress_zscore"] = (r["demand_supply_ratio"] - mu) / sd if sd else 0.0

print(f"Months: {len(records)} | Mean ratio: {mu:.6f} | SD: {sd:.6f}")
records[:2]


In [ ]:
# Figure 1: Consumption vs Supply
x = [r["date"] for r in records]
write_svg_line_chart(
    FIG_DIR / "stress_consumption_vs_supply.svg",
    "Total Sector: Consumption vs Supply",
    x,
    {
        "consumption": [r["consumption"] for r in records],
        "supply": [r["supply"] for r in records],
    },
    y_label="MWh",
)

# Figure 2: Demand/Supply ratio
write_svg_line_chart(
    FIG_DIR / "stress_ratio.svg",
    "Demand-Supply Ratio (Consumption / Supply)",
    x,
    {"ratio": [r["demand_supply_ratio"] for r in records]},
    y_label="ratio",
)

# Figure 3: Stress z-score bars
write_svg_bar_chart(
    FIG_DIR / "stress_zscore_top_months.svg",
    "Top 15 Stress Months (z-score)",
    [r["date"] for r in sorted(records, key=lambda x: x["stress_zscore"], reverse=True)[:15]],
    [r["stress_zscore"] for r in sorted(records, key=lambda x: x["stress_zscore"], reverse=True)[:15]],
    y_label="z-score",
)

print("Saved figures to:")
for p in sorted(FIG_DIR.glob("stress_*.svg")):
    print("-", p)


In [ ]:
top10 = sorted(records, key=lambda x: x["stress_zscore"], reverse=True)[:10]
print("Top 10 high-stress months")
print("date       | ratio     | reserve_proxy | zscore")
for r in top10:
    print(f"{r['date']} | {r['demand_supply_ratio']:.6f} | {r['reserve_proxy']:12.6f} | {r['stress_zscore']:.3f}")
